In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Necessary modules are imported

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib.pyplot import figure

# Importing data

In [ ]:
india=pd.read_csv("../input/suicides-in-india/Suicides in India 2001-2012.csv")
india.head(20)

In [ ]:
#Dropping null values
india['Total']=india['Total'].replace(0, np.nan)

india=india.drop(india[india['Total'].isna()].index)

# Preparing data for EDA

In [ ]:
cause=india[india['Type_code']=='Causes'].reset_index(drop=True)
cause=cause.drop("Type_code", axis=1)
print(cause)

# Distribution of suicides in various domains

In [ ]:
# Our dataset

cause

# Distribution of cases as per the age group

In [ ]:
def Age_group(age):
    sum=cause[cause["Age_group"]==age]['Total'].sum()
    return sum

age_group=["0-14", "15-29", "30-44", "45-59", "60+"]
value=[]
for age in age_group:
    #dict[age]=Age_group(age)
    value.append(Age_group(age))


#age=pd.DataFrame.from_dict(dict, orient='index')
age=pd.DataFrame({"age":age_group,"count":value})
sns.catplot(x="age", y="count", data=age, kind="bar")
age

# Distribution as per the gender

In [ ]:
def Gender(sex):
    sum=cause[cause["Gender"]==sex]['Total'].sum()
    return sum

gen=["Male", "Female"]
list=[]
for sex in gen:
    list.append(Gender(sex))

gender=pd.DataFrame({"Gender":gen,"count":list})
sns.catplot(x="Gender", y="count", data=gender, kind="bar")
gender

# Distribution in different states

In [ ]:
def State_wise(state):
    sum=cause[cause["State"]==state]['Total'].sum()
    return sum

states=cause["State"].unique()
state=[]
for st in states:
    #dict[age]=Age_group(age)
    state.append(State_wise(st))


#age=pd.DataFrame.from_dict(dict, orient='index')
state=pd.DataFrame({"state":states,"count":state})
fig_dims = (8, 6)

fig, ax = plt.subplots(figsize=fig_dims)

sns.barplot(x = "state", y = "count", ax=ax, data=state)
#sns.catplot(x="state", y="count", data=state, kind="bar")
plt.xticks(rotation=90)
state

# Year Wise Distribution

In [ ]:
def Year_wise(year):
    sum=cause[cause["Year"]==year]['Total'].sum()
    return sum

years=cause["Year"].unique()
year=[]
for yr in years:
    #dict[age]=Age_group(age)
    year.append(Year_wise(yr))

print(year)
#age=pd.DataFrame.from_dict(dict, orient='index')
year=pd.DataFrame({"year":years,"count":year})
fig_dims = (10, 8)

fig, ax = plt.subplots(figsize=fig_dims)

sns.barplot(x = "year", y = "count", ax=ax, data=year)
#sns.catplot(x="state", y="count", data=state, kind="bar")

Suicides due to various causes

In [ ]:
def cause_wise(cs):
    sum=cause[cause["Type"]==cs]['Total'].sum()
    return sum

causes=cause["Type"].unique()
reason=[]

for cs in causes:
    #dict[age]=Age_group(age)
    reason.append(cause_wise(cs))

#age=pd.DataFrame.from_dict(dict, orient='index')
reason=pd.DataFrame({"cause":causes,"count":reason})
fig_dims = (8, 6)

fig, ax = plt.subplots(figsize=fig_dims)

sns.barplot(x = "cause", y = "count", ax=ax, data=reason)
#sns.catplot(x="state", y="count", data=state, kind="bar")
plt.xticks(rotation=90)
reason

# Preparing data to be fed into the model

In [ ]:
#Encoding column type using Label Encoder

from sklearn.preprocessing import LabelEncoder

lb_make=LabelEncoder()

cause["Type"]=lb_make.fit_transform(cause["Type"])

In [ ]:
#Encoding male and female

cause['Gender']=cause['Gender'].replace("Female",0)
cause['Gender']=cause['Gender'].replace("Male",1)

In [ ]:
#Creating dummies of the age group

dummies = pd.get_dummies(cause[['Age_group']])
cause_dummy= pd.concat([cause.drop(['Age_group'],axis=1), dummies],axis=1)
cause=cause_dummy

In [ ]:
#Encoding each year as per the importance

cause.replace([2001,2002,2003], 0, inplace=True)
cause.replace([2004,2005], 1, inplace=True)
cause.replace([2006,2012], 2, inplace=True)
cause.replace([2007,2008], 3, inplace=True)
cause.replace(2009, 4, inplace=True)
cause.replace([2010,2011], 5, inplace=True)

In [ ]:
#Encoding each year as per the cases

cause.replace(['A & N Islands','Arunachal Pradesh','Chandigarh','D & N Haveli','Daman & Diu','Goa','Himachal Pradesh','Jammu & Kashmir','Lakshadweep','Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Sikkim','Uttarakhand'],1,inplace=True)
cause.replace(['Bihar','Jharkhand','Puducherry','Punjab','Tripura'],2, inplace=True)
cause.replace(['Delhi (Ut)'],3,inplace=True)
cause.replace(['Assam','Haryana'],4,inplace=True)
cause.replace(['Odisha', 'Rajasthan','Uttar Pradesh'],5,inplace=True)
cause.replace(['Chhattisgarh','Gujarat'],6,inplace=True)
cause.replace(['Madhya Pradesh'],7,inplace=True)
cause.replace(['Kerala'],8,inplace=True)
cause.replace(['Andhra Pradesh','Tamil Nadu', 'West Bengal'],10,inplace=True)
cause.replace(["Karnataka"],9,inplace=True)
cause.replace(["Maharashtra"],10,inplace=True)
cause

In [ ]:
#Using Statistical Analysis to find out the significant features

import statsmodels.api as sm

col=["State", "Year", "Gender", "Type",'Age_group_0-14', 'Age_group_15-29', 'Age_group_30-44', 	'Age_group_45-59', 'Age_group_60+']
print(col)
mod=sm.OLS(cause['Total'], cause[col])

re=mod.fit()

print(re.summary())

In [ ]:
from sklearn import metrics as sm

y=cause["Total"]
x=cause[col]
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(x,y,test_size=0.25,random_state=56)

reg=LinearRegression()
reg.fit(X_train, y_train)
pred=reg.predict(X_test)
predict=pd.DataFrame(pred)

print(reg.score(X_test, y_test))
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, predict), 2))
print("Mean squared error =", round(sm.mean_squared_error(y_test, predict), 2)) 
print("R2 score =", round(sm.r2_score(y_test, predict), 2))

In [ ]:
importance = reg.coef_
# summarize feature importance
for i,v in enumerate(importance):
	print('Linear Regression Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

# Maximum accuracy is achieved when we are using Bootstrap aggregation

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

      
tree=DecisionTreeRegressor(max_depth=11, random_state=1)
bag=BaggingRegressor(base_estimator=tree, n_estimators=100, random_state=1)

tree.fit(X_train, y_train)
print("Accuracy of DecisionTree regressor",tree.score(X_test, y_test))

bag.fit(X_train, y_train)

print("Accuracy of bagging regressor", bag.score(X_test, y_test))

In [ ]:
importance = tree.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Decision Tree Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()